<a href="https://colab.research.google.com/github/Kaveeshala/Hatchlings_monitoring_model/blob/main/Train_Behavior.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# CELL 2
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Dropout

# 1. Define the AI Architecture
model = Sequential([
    # Input: 30 steps of (x,y) coordinates
    GRU(64, input_shape=(30, 2), return_sequences=False),
    Dropout(0.2),  # Prevents over-memorizing
    Dense(32, activation='relu'),
    # Output: 3 probabilities (Normal, Floater, Circler)
    Dense(3, activation='softmax')
])

# 2. Compile
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# 3. Train
print("Training Behavior Model...")
history = model.fit(X, y, epochs=20, batch_size=32, validation_split=0.2)

# 4. Save the Brain
model.save("behavior_gru.h5")
print("\n Model Saved as 'behavior_gru.h5'")

/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Training Behavior Model...
Epoch 1/20
75/75 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - accuracy: 0.3589 - loss: 1.0969 - val_accuracy: 0.3883 - val_loss: 1.0929
Epoch 2/20
75/75 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.3965 - loss: 1.0869 - val_accuracy: 0.3733 - val_loss: 1.0869
Epoch 3/20
75/75 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.4038 - loss: 1.0741 - val_accuracy: 0.3517 - val_loss: 1.0929
Epoch 4/20
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.4105 - loss: 1.0618 - val_accuracy: 0.3733 - val_loss: 1.0695
Epoch 5/20
75/75 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.4738 - loss: 1.0189 - val_accuracy: 0.5017 - val_loss: 1.0173
Epoch 6/20
75/75 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.6457 - loss: 0.8915 - val_accuracy: 0.7283 - val_loss: 0.7643
Epoch 7/20
75/75 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.7375 - loss: 0.6985 - val_accuracy: 0.7867 - val_loss: 0.6441
Epoch 8/20
75/75 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.7955 - loss: 0.5918 - val_


 Model Saved as 'behavior_gru.h5'


In [1]:
#GENERATE SYNTHETIC DATA
import numpy as np
import os

# 1. Setup
NUM_SAMPLES = 3000      # 1000 Normal, 1000 Floaters, 1000 Circlers
SEQ_LENGTH = 30         # 30 Frames (1 second of video)
TANK_SIZE = 640         # Resolution of your video

# 2. Define the 3 Patterns
def generate_normal_path():
    # Active swimmer: Long straight lines, bouncing off walls
    path = []
    x, y = np.random.randint(50, TANK_SIZE-50), np.random.randint(50, TANK_SIZE-50)
    speed = np.random.uniform(8, 15)
    angle = np.random.uniform(0, 2 * np.pi)
    for _ in range(SEQ_LENGTH):
        path.append([x, y])
        x += speed * np.cos(angle); y += speed * np.sin(angle)
        if x < 10 or x > TANK_SIZE-10: angle = np.pi - angle
        if y < 10 or y > TANK_SIZE-10: angle = -angle
    return np.array(path)

def generate_floater_path():
    # Sick/Buoyant: Drifting slowly, random jitter
    path = []
    x, y = np.random.randint(100, TANK_SIZE-100), np.random.randint(100, TANK_SIZE-100)
    speed = np.random.uniform(0, 2)
    for _ in range(SEQ_LENGTH):
        path.append([x, y])
        angle = np.random.uniform(0, 2 * np.pi) # Random direction every step
        x += speed * np.cos(angle); y += speed * np.sin(angle)
    return np.array(path)

def generate_circler_path():
    # Disoriented: Spinning in tight circles
    path = []
    cx, cy = np.random.randint(100, TANK_SIZE-100), np.random.randint(100, TANK_SIZE-100)
    radius = np.random.uniform(20, 50)
    speed = np.random.uniform(0.1, 0.3)
    angle = 0
    for _ in range(SEQ_LENGTH):
        x = cx + radius * np.cos(angle); y = cy + radius * np.sin(angle)
        path.append([x, y])
        angle += speed
    return np.array(path)

# 3. Create Dataset
X_data, y_labels = [], []
for i in range(NUM_SAMPLES):
    if i < 1000:
        X_data.append(generate_normal_path()); y_labels.append(0) # 0 = Normal
    elif i < 2000:
        X_data.append(generate_floater_path()); y_labels.append(1) # 1 = Floater
    else:
        X_data.append(generate_circler_path()); y_labels.append(2) # 2 = Circler

# 4. Normalize and Save
X = np.array(X_data) / TANK_SIZE # Squash numbers between 0 and 1
y = np.array(y_labels)

# Shuffle
idx = np.arange(len(X)); np.random.shuffle(idx); X, y = X[idx], y[idx]

print(f"Data Generated! Shape: {X.shape}")

Data Generated! Shape: (3000, 30, 2)
